# Dataset Preparation Tutorial

Welcome to the dataset preparation tutorial! In this notebook, we will download the toy data set for the tutorial and prepare the necessary tables used for later analysis. Here are the steps we will review:

1. Verify prerequisites
2. Create a new project workspace
3. Review sample dataset
4. Build the proxy table
5. Run regional annotation ETL

**NOTE**: All of the configuration files for this tutorial have been provided in the container. The host and port values in the configuration files are dynamically set based on your system. 

**NOTE**: The current working directory is '~/vmount/notebooks'. All file and directory paths specified in the configuration files are relative to the current working directory. 

## 1. Verify prerequisites

Here are the software prerequisites for executing tasks with luna packages. These prerequisites have already been baked into this docker container. To view the setup, please see the corresponding dockerfile. 

In [1]:
!python3 --version
!echo LUNA_HOME: $LUNA_HOME
import luna.pathology
print(luna.pathology.__path__)

Python 3.9.16
LUNA_HOME: /home/pollardw/vmount
['/opt/conda/lib/python3.9/site-packages/luna/pathology']


## 2. Create a new project workspace



Next, we create a luna home space and place the configuration files there. Using a manifest file, we will create a project workspace where your configurations, data, models, and outputs will go for this tutorial.

In [2]:
%%bash
mkdir -p ~/luna
cp -R ~/vmount/conf ~/luna
cat ~/luna/conf/manifest.yaml
cp ~/luna/conf/manifest.yaml "${LUNA_HOME}/PRO-12-123/"

# project manifest template

# MIND project id
PROJECT: PRO-12-123

# IRB
IRB:

# project title
TITLE: pathology-tutorial

# project description
DESCRIPTION: End-to-end pathology analysis tutorial

DATA_MODALITIES: pathology

ROOT_PATH: ../


You should now see the `manifest.yaml` file in your `vmount/PRO-12-123` directory.  This will be your project workspace.

## 3. Review sample dataset

The data we'll be using for this tutorial is a set of 5 whole slide images (WSI) of ovarian cancer H&E slides, available in the svs file format. Whole slide imaging refers to the scanning of conventional glass slides for research purposes; in this case, these are slides that oncologists have used while inspecting cancer samples.

The slides have been downloaded by the script `vmount/provision_girder.py`, which ...
  
  - Creates an admin user and default assetstore
  
  - Downloads sample data from [public kitware site](https://data.kitware.com/#user/61b9f3dc4acac99f42ca7678/folder/61b9f4564acac99f42ca7692). to `~/vmount/PRO-12-123/data/toy_data_set/`
  
  - Creates a collection and adds the slides to your local DSA
    
If this was successsful, the downloaded svs files will be listed by the `tree` command, below ... 

In [3]:
!tree "${LUNA_HOME}/PRO-12-123/data/toy_data_set"

/home/pollardw/vmount/PRO-12-123/data/toy_data_set
├── 01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs
├── 01OV002-ed65cf94-8bc6-492b-9149-adc16f.svs
├── 01OV007-9b90eb78-2f50-4aeb-b010-d642f9.svs
├── 01OV008-308ad404-7079-4ff8-8232-12ee2e.svs
├── 01OV008-7579323e-2fae-43a9-b00f-a15c28.svs
└── table

1 directory, 5 files


These files may also be viewed through the Minio instance. The URL for this instance is available in the docker-compose terminal log. 

If you want to import your own data, you can do so from your local filesystem as well as an object store. For more details, refer to the [girder user documentation](https://girder.readthedocs.io/en/latest/user-guide.html#assetstores)

To import images from your local filesystem, 

- Login to DSA with admin/password1
- Navigate to **Collections** and use the create collection interface to create a new collection
- Under the branch icon on the right, create a folder within the collection
- In this folder, use the blue **info** button to access the Unique ID for the folder, and copy and store it somewhere for reference.
- Add images to your local computer at `vmount/assetstore` 
- Navigate to **Admin Console** -> **Assetstores**
- From the default assetstore, click on **Import data**
- Specify the path to the images you wish to import using absolute path, e.g. `/home/<user>/vmount/assetstore/yourimage`, and specify the destination type as 'Folder' and the destination ID as the ID copied earlier, and click import

As the `/assetstore` mount is available to DSA, this import should be much faster than uploading the image through the **Upload files** button in the UI.


## 4. Build the proxy table

Now, we will run the Whole Slide Image (WSI) ETL to build a meta-data catalog of the slides in a proxy table. 

For reference, ETL stands for extract-transform-load; it is the method that often involves cleaning data, transforming data types, and loading data into different systems. 

In [4]:
!slide_etl "${LUNA_HOME}/PRO-12-123/data/toy_data_set" \
--project_name PRO-12-123 --comment "Example Ingestion Job" \
--no-copy --output-urlpath "${LUNA_HOME}/PRO-12-123/data/toy_data_set/table/SLIDES"

2023-07-24 16:15:28.422 | INFO     | luna.pathology.cli.slide_etl:cli:93 -                                        id  ... properties.tiffslide.level[3].downsample
0  01OV002-bd8cdc70-3d46-40ae-99c4-90ef77  ...                                32.007005
1  01OV002-ed65cf94-8bc6-492b-9149-adc16f  ...                                32.008682
2  01OV007-9b90eb78-2f50-4aeb-b010-d642f9  ...                                32.003727
3  01OV008-308ad404-7079-4ff8-8232-12ee2e  ...                                      NaN
4  01OV008-7579323e-2fae-43a9-b00f-a15c28  ...                                32.002432

[5 rows x 70 columns]
2023-07-24 16:15:28.480 | INFO     | luna.pathology.cli.slide_etl:cli:105 - Writing to parquet file
2023-07-24 16:15:28.503 | DEBUG    | luna.common.utils:wrapper:127 - cli ran in 2.34s


This step may take a while. At the end, your proxy table should be generated!

**Note** code for the following text has been handled elsewhere, but it is still good to read the following to familiarize yourself with some potential issues with overfitting data.

Before we view the table, we must first update it to associate patient IDs with the slides. This is necessary for correctly training and validating the machine learning model in the coming notebooks. Once the slides are divided into "tiles" in the next notebook, the tiles are split between the training and validation sets for the ML model. If the tiles do not have patient ID's associated with them, then it is possible for tiles from one individual to appear in both the training and validation of the model; this would cause researchers to have an exaggerated interpretation of the model's accuracy, since we would essentially be validating the model on information that is too near to what it has already seen. 

Note that we will not be using patient IDs associated with MSK. Instead, we will be using spoof IDs that will suffice for this tutorial. When running this workflow with real data, make sure to include the IDs safely and securely. Run the following block of code to add a 'patient_id' column to the table and store it using Spark (DEPRECATED).

Next, we may view the WSI table! This table should have the metadata associated with the WSI slides that you just collected, including the patient IDs. 

This table may also be viewed through the Dremio instance. The URL for this instance is available in the docker-compose terminal log. 

In [5]:
import os
LUNA_HOME = os.environ["LUNA_HOME"]
TABLE_DIR = f"{LUNA_HOME}/PRO-12-123/data/toy_data_set/table"

import pandas as pd
pd.read_parquet(f"{TABLE_DIR}/SLIDES/slide_ingest_PRO-12-123.parquet")

,id,project_name,comment,slide_size,uuid,url,channel0_R,channel0_G,channel0_B,channel1_R,...,properties.tiffslide.mpp-y,properties.tiff.ImageDescription,properties.tiffslide.vendor,properties.aperio.Focus Offset,properties.aperio.ImageID,properties.tiffslide.mpp-x,properties.tiffslide.level[3].tile-height,properties.aperio.LineAreaYOffset,properties.tiffslide.level[2].height,properties.tiffslide.level[3].downsample
0,01OV002-bd8cdc70-3d46-40ae-99c4-90ef77,PRO-12-123,Example Ingestion Job,237047223,cd8c82fb-69d4-3ef2-8cb7-1eb0919c8d36,file:///home/pollardw/vmount/PRO-12-123/data/t...,0.620722,0.651500,0.436179,0.344821,...,0.2501,Aperio Image Library vFS90 01\r\n56000x54940 [...,aperio,0.0,109316,0.2501,256.0,0.004639,3427,32.007005
1,01OV002-ed65cf94-8bc6-492b-9149-adc16f,PRO-12-123,Example Ingestion Job,240691747,e60d567c-a5d6-3030-b890-49f454eeee6c,file:///home/pollardw/vmount/PRO-12-123/data/t...,0.628632,0.613612,0.477809,0.386583,...,0.2501,Aperio Image Library vFS90 01\r\n53000x49887 [...,aperio,0.0,109327,0.2501,256.0,0.004639,3111,32.008682
2,01OV007-9b90eb78-2f50-4aeb-b010-d642f9,PRO-12-123,Example Ingestion Job,262796337,71a882f0-b708-3a57-b152-b5ddfffe84ba,file:///home/pollardw/vmount/PRO-12-123/data/t...,0.571893,0.659842,0.487388,0.268574,...,0.2501,Aperio Image Library vFS90 01\r\n114000x60210 ...,aperio,0.0,109328,0.2501,256.0,0.004639,3756,32.003727
3,01OV008-308ad404-7079-4ff8-8232-12ee2e,PRO-12-123,Example Ingestion Job,207479411,93604598-ede6-312b-b060-5375a9823e4b,file:///home/pollardw/vmount/PRO-12-123/data/t...,0.585755,0.675688,0.447589,0.343329,...,0.2501,Aperio Image Library vFS90 01\r\n55000x38728 [...,aperio,0.0,109330,0.2501,NaN,0.004639,2414,NaN
4,01OV008-7579323e-2fae-43a9-b00f-a15c28,PRO-12-123,Example Ingestion Job,215796305,ae824866-e870-32b9-90d2-b261fe78ce94,file:///home/pollardw/vmount/PRO-12-123/data/t...,0.618213,0.666116,0.417255,0.267862,...,0.2501,Aperio Image Library vFS90 01\r\n44000x59309 [...,aperio,0.0,109331,0.2501,256.0,0.004639,3700,32.002432


If the table loads, then you have successfully run the Whole Slide Image (WSI) ETL to database the slides.

## Run the regional annotation ETL

The whole slide images that you downloaded are images of ovarian cancer, but not every pixel on each slide is a tumor.  In fact, the images typically include tumor cells, normal ovarian cells, lymphoctyes, and more. *Note: A non-expert annotated this slide for demo purposes only.*

The regional annotation ETL performs the following steps

- Download DSA json annotations
- Convert DSA jsons to GeoJSON format, which is compatible with downstream applications
- Save configs in your `~/vmount/PRO-12-123/configs/REGIONAL_METADATA_RESULTS`
- Save parquet table in your `~/vmount/PRO-12-123/tables/REGIONAL_METADATA_RESULTS `


To run the regional annotation ETL, we use the `dsa_annotation` CLI. For more details on the `dsa_annotation` tool, and the annotations we support, please checkout the `7_dsa-annotation.ipynb` notebook.


In [6]:
!mkdir -p "{TABLE_DIR}/ANNOTATIONS"
!dsa_annotation http://girder:8080/api/v1 \
    --collection-name "TCGA collection" \
    --annotation-name "ov_regional" \
    --username admin --password password1 \
    --output-urlpath "{TABLE_DIR}/ANNOTATIONS"

2023-07-24 16:15:46.623 | INFO     | luna.pathology.cli.dsa_annotation_etl:dsa_annotation_etl:144 - Dashboard: http://192.168.128.6:8787/status
                                                             _id  ... annotation_name
slide_id                                                          ...                
01OV002-bd8cdc70-3d46-40ae-99c4-90ef77  64be9d70d34214cefc267e70  ...     ov_regional
01OV002-bd8cdc70-3d46-40ae-99c4-90ef77  64be9d70d34214cefc267e70  ...     ov_regional
01OV002-bd8cdc70-3d46-40ae-99c4-90ef77  64be9d70d34214cefc267e70  ...     ov_regional
01OV002-bd8cdc70-3d46-40ae-99c4-90ef77  64be9d70d34214cefc267e70  ...     ov_regional
01OV002-bd8cdc70-3d46-40ae-99c4-90ef77  64be9d70d34214cefc267e70  ...     ov_regional
01OV002-bd8cdc70-3d46-40ae-99c4-90ef77  64be9d70d34214cefc267e70  ...     ov_regional
01OV002-bd8cdc70-3d46-40ae-99c4-90ef77  64be9d70d34214cefc267e70  ...     ov_regional
01OV002-ed65cf94-8bc6-492b-9149-adc16f  64be9d7dd34214cefc267e7c  ...     ov_regio

To check that the regional annotation ETL was correctly run, you can examine the regional annotations table.  This table contains the metadata saved by the ETL.  It includes paths to the bitmap files, numpy files, and geoJSON files that were mentioned before.  To load the table, run the following cell,

In [7]:
from pyarrow.parquet import read_table
annotations_table = f"{TABLE_DIR}/ANNOTATIONS/slide_annotation_dataset_TCGA collection_ov_regional.parquet"
pd.read_parquet(annotations_table)

,_id,baseParentId,baseParentType,created,creatorId,description,folderId,largeImage,lowerName,name,...,xmax,ymin,ymax,bbox_area,x_coords,y_coords,slide_geojson,collection_uuid,collection_name,annotation_name
slide_id,,,,,,,,,,,,,,,,,,,,,
01OV002-bd8cdc70-3d46-40ae-99c4-90ef77,64be9d70d34214cefc267e70,64be9d6fd34214cefc267e6e,collection,2023-07-24T15:49:04.351000+00:00,64be9d6fd34214cefc267e6a,,64be9d6fd34214cefc267e6f,"{'fileId': '64be9d7cd34214cefc267e72', 'source...",01ov002-bd8cdc70-3d46-40ae-99c4-90ef77.svs,01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs,...,28661.0,40529.0,44372.0,13108473.0,"[28211, 28328, 28587, 28630, 28655, 28661, 286...","[42225, 42546, 43126, 43261, 43379, 43607, 437...",None,64be9d6fd34214cefc267e6e,TCGA collection,ov_regional
01OV002-bd8cdc70-3d46-40ae-99c4-90ef77,64be9d70d34214cefc267e70,64be9d6fd34214cefc267e6e,collection,2023-07-24T15:49:04.351000+00:00,64be9d6fd34214cefc267e6a,,64be9d6fd34214cefc267e6f,"{'fileId': '64be9d7cd34214cefc267e72', 'source...",01ov002-bd8cdc70-3d46-40ae-99c4-90ef77.svs,01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs,...,33932.0,35713.0,39793.0,9792000.0,"[32252, 32252, 32220, 32164, 32140, 32108, 320...","[37097, 37001, 36897, 36745, 36713, 36689, 366...",None,64be9d6fd34214cefc267e6e,TCGA collection,ov_regional
01OV002-bd8cdc70-3d46-40ae-99c4-90ef77,64be9d70d34214cefc267e70,64be9d6fd34214cefc267e6e,collection,2023-07-24T15:49:04.351000+00:00,64be9d6fd34214cefc267e6a,,64be9d6fd34214cefc267e6f,"{'fileId': '64be9d7cd34214cefc267e72', 'source...",01ov002-bd8cdc70-3d46-40ae-99c4-90ef77.svs,01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs,...,25542.0,18922.0,21735.0,5583805.0,"[24180, 24136, 24063, 23980, 23874, 23813, 237...","[18972, 18978, 19017, 19078, 19161, 19217, 192...",None,64be9d6fd34214cefc267e6e,TCGA collection,ov_regional
01OV002-bd8cdc70-3d46-40ae-99c4-90ef77,64be9d70d34214cefc267e70,64be9d6fd34214cefc267e6e,collection,2023-07-24T15:49:04.351000+00:00,64be9d6fd34214cefc267e6a,,64be9d6fd34214cefc267e6f,"{'fileId': '64be9d7cd34214cefc267e72', 'source...",01ov002-bd8cdc70-3d46-40ae-99c4-90ef77.svs,01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs,...,30651.0,23202.0,26990.0,14015600.0,"[30133, 30073, 30005, 29951, 29904, 29857, 298...","[26411, 26465, 26506, 26532, 26553, 26566, 265...",None,64be9d6fd34214cefc267e6e,TCGA collection,ov_regional
01OV002-bd8cdc70-3d46-40ae-99c4-90ef77,64be9d70d34214cefc267e70,64be9d6fd34214cefc267e6e,collection,2023-07-24T15:49:04.351000+00:00,64be9d6fd34214cefc267e6a,,64be9d6fd34214cefc267e6f,"{'fileId': '64be9d7cd34214cefc267e72', 'source...",01ov002-bd8cdc70-3d46-40ae-99c4-90ef77.svs,01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs,...,17365.0,26776.0,29672.0,11097472.0,"[16525, 16597, 16701, 16781, 16845, 16901, 169...","[27088, 26992, 26896, 26840, 26816, 26808, 267...",None,64be9d6fd34214cefc267e6e,TCGA collection,ov_regional
01OV002-bd8cdc70-3d46-40ae-99c4-90ef77,64be9d70d34214cefc267e70,64be9d6fd34214cefc267e6e,collection,2023-07-24T15:49:04.351000+00:00,64be9d6fd34214cefc267e6a,,64be9d6fd34214cefc267e6f,"{'fileId': '64be9d7cd34214cefc267e72', 'source...",01ov002-bd8cdc70-3d46-40ae-99c4-90ef77.svs,01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs,...,23500.0,16457.0,19848.0,6921031.0,"[23389, 23435, 23463, 23481, 23500, 23500, 234...","[18929, 19021, 19103, 19186, 19250, 19425, 194...",None,64be9d6fd34214cefc267e6e,TCGA collection,ov_regional
01OV002-bd8cdc70-3d46-40ae-99c4-90ef77,64be9d70d34214cefc267e70,64be9d6fd34214cefc267e6e,collection,2023-07-24T15:49:04.351000+00:00,64be9d6fd34214cefc267e6a,,64be9d6fd34214cefc267e6f,"{'fileId': '64be9d7cd34214cefc267e72', 'source...",01ov002-bd8cdc70-3d46-40ae-99c4-90ef77.svs,01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs,...,NaN,NaN,NaN,NaN,None,None,/home/pollardw/vmount/PRO-12-123/data/toy_data...,64be9d6fd34214cefc267e6e,TCGA collection,ov_regional
01OV002-ed65cf94-8bc6-492b-9149-adc16f,64be9d7dd34214cefc267e7c,64be9d6fd34214cefc267e6e,collection,2023-07-24T15:49:17.982000+00:00,64be9d6fd34214cefc267e6a,,64be9d6fd34214

Lastly, lets get our geojsons and join on slide ID.

In [8]:
pd.read_parquet(annotations_table) \
    .query("type=='geojson'")[['slide_geojson']] \
    .join(
        pd.read_parquet(f"{TABLE_DIR}/SLIDES/slide_ingest_PRO-12-123.parquet")['id']
    )

,slide_geojson,id
slide_id,,
01OV002-bd8cdc70-3d46-40ae-99c4-90ef77,/home/pollardw/vmount/PRO-12-123/data/toy_data...,NaN
01OV002-ed65cf94-8bc6-492b-9149-adc16f,/home/pollardw/vmount/PRO-12-123/data/toy_data...,NaN
01OV007-9b90eb78-2f50-4aeb-b010-d642f9,/home/pollardw/vmount/PRO-12-123/data/toy_data...,NaN
01OV008-308ad404-7079-4ff8-8232-12ee2e,/home/pollardw/vmount/PRO-12-123/data/toy_data...,NaN
01OV008-7579323e-2fae-43a9-b00f-a15c28,/home/pollardw/vmount/PRO-12-123/data/toy_data...,NaN


At this point, you have successfully set up your workspace, downloaded the data, and run both the pathology and regional annotation ETLs to prepare your data. You are ready to move on to the tiling notebook!